In [3]:
import pandas as pd
import os

In [258]:
KEGG_DB_LOC = '/data11/bio/databases/kegg/'
GEN_LOC = '/data11/bio/databases/KEGG_HUMANN2_BREWED'

In [256]:
ko_pathway_list_loc = os.path.join(KEGG_DB_LOC, 'genes/ko/ko_pathway.list')

if os.path.isfile(ko_pathway_list_loc):
    ko_pathway_pd = pd.read_csv(ko_pathway_list_loc, header = None, sep = '\t')
    
ko_pathway_only_ko_pd = ko_pathway_pd.loc[ko_pathway_pd[1].str.split(':').str[1].str.startswith('ko')]
ko_pathway_only_ko_pd.loc[:, 0] = ko_pathway_only_ko_pd[0].str.replace('ko:', '')
ko_pathway_only_ko_pd.loc[:, 1] = ko_pathway_only_ko_pd[1].str.replace('path:', '')

kos = ko_pathway_only_ko_pd[1].unique()
kos_w_K = {}
for ko in kos:
    sub = ko_pathway_only_ko_pd.loc[ko_pathway_only_ko_pd[1]== ko]
    Ks = sub[0]
    kos_w_K.update({ko: list(Ks)})
    
str_to_write = ''

for k,v in kos_w_K.items():
    str_to_write = str_to_write + k + '\t'
    str_to_write += '\t'.join(v)
    str_to_write += '\n'
str_to_write = str_to_write[0:-1]

/home/fedorov/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [259]:
with open(os.path.join(GEN_LOC,'keggc'), 'x') as keggc_file:
    keggc_file.write(str_to_write)

In [253]:
def parse_ko(lines):
    entry = ''
    # First line conatins ENTRY, last line is ///
    line = lines[0]
    curr_line_index = 0
    while curr_line_index < len(lines):
        line = lines[curr_line_index]
        if line.startswith('ENTRY'): 
            entry = line.replace('ENTRY', '').replace('KO', '').strip(' ')
            curr_line_index += 1
        elif line.startswith('NAME'): 
            name = line
            curr_line_index += 1

        elif line.startswith('GENES'):
            last_gene_index = curr_line_index+1
            # All gene lines start with whitespace, grab them
            while lines[last_gene_index][0] == ' ':
                last_gene_index += 1

            genes = lines[curr_line_index:last_gene_index]
            if len(genes) == 0:
                print('!!!!!!!!!!!!! ZERO GENES')
                print('hhj', lines[curr_line_index])
                print(lines[last_gene_index - 1])
            else:
                genes[0] = genes[0].replace('GENES', '')
                genes = [g.strip(' ') for g in genes]
                ggs = []
                
                str_to_write_full = ''
                
                for g in genes:
                    organism = g.split(':')[0]
                    gene_full =  g.split(':')[1].strip(' ')
                    genes_splitted = gene_full.split(' ')
                    
                    only_gene_names = [ggg.split('(')[0] for ggg in genes_splitted] 
                    for gene in only_gene_names:
                        str_to_write_full = str_to_write_full + organism +'#'+gene+'\t'
                        
                ggs = {'ENTRY': entry, 'FULL_STR': str_to_write_full}
            curr_line_index = last_gene_index
        else:
            curr_line_index += 1
            
    return ggs


In [254]:
KOS = []
org_genes = []
with open('/data11/bio/databases/kegg/genes/ko/ko', 'r') as ko:
    line = ko.readline().strip('\n')
    while line:
        if line.startswith('ENTRY'): 
            ko_lines = [line, ko.readline().strip('\n')]
            while not ko_lines[-1].startswith('///'):
                ko_lines.append(ko.readline().strip('\n'))
            ggs = parse_ko(ko_lines)
            org_genes.append(ggs)
        line = ko.readline().strip('\n')
#         break
    
# ggs = pd.concat(org_genes)

In [255]:
with open('/data11/bio/databases/KEGG_HUMANN2_BREWED/koc', 'w') as KOC:
    entries_num = len(org_genes)
    for i, entry in enumerate(org_genes):
        full_str = entry['ENTRY'] + '\t' + entry['FULL_STR'].rstrip('\t')
        if i != entries_num - 1:
            full_str += '\n'
        KOC.write(full_str)

In [228]:
start = time.time()
sub = ggs_wo_rna.loc[ggs_wo_rna['ENTRY'] == entry]
end = time.time()
print(end-start)

0.5527212619781494


In [233]:
sub.groupby('ENTRY').get_group('str_to_write')

KeyError: 'str_to_write'

In [232]:
sub

,organism,gene_full,gene,ENTRY,str_to_write
0,HSA,7358(UGDH),7358,K00012,HSA#7358
1,PTR,471171(UGDH),471171,K00012,PTR#471171
2,PPS,100976107(UGDH),100976107,K00012,PPS#100976107
3,GGO,101137330(UGDH),101137330,K00012,GGO#101137330
4,PON,100174620(UGDH),100174620,K00012,PON#100174620
...,...,...,...,...,...
6080,VG,5364530(N712L),5364530,K00012,VG#5364530
6081,VG,5470943(Z571L),5470943,K00012,VG#5470943
6082,VG,5660396(C729L),5660396,K00012,VG#5660396
6083,VG,918088(A609L),918088,K00012,VG#918088


In [205]:
entry +'\t' + '\t'.join(sub['str_to_write']) +'\n'

'K00533\tCAC#CA_C0028\tCAE#SMB_G0028\tCAY#CEA_G0028\tCPE#CPE2346\tCPF#CPF_2655\tCPR#CPR_2341\tCNO#NT01CX_2259\tCBK#CLL_A0052\tCBN#CbC4_1525\tCBT#CLH_0034\tCKL#CKL_2257\tCKR#CKR_1986\tCCB#Clocel_4097\tCPAS#Clopa_0056\tCPAT#CLPA_c00280\tCPAE#CPAST_c00280\tCSB#CLSA_c00340\tCLT#CM240_0483\tCBV#U729_1141\tCBUT#ATN24_01590\tCTYK#CTK_C05160\tCTAE#BGI42_00165\tCCHV#BTM20_00205\tCIA#BEN51_00210\tCSEP#CP523_07965\tCDY#F3K33_11495\tASF#SFBM_0914\tASM#MOUSESFB_0852\tASO#SFBmNL_00989\tASB#RATSFB_0771\tAG#AAA23248\n'